# ECBM 4040 Final Project - Group ERP1 Notebook 1

In [4]:
# !pip install opencv-python


     |████████████████████████████████| 52.4MB 276kB/s eta 0:00:01     |█████████▋                      | 15.7MB 851kB/s eta 0:00:44


In [4]:
 !pip install imgaug

In [1]:
import sys
import os
from PIL import Image

import numpy as np
import sklearn
import tensorflow as tf
import tensorflow.keras
from keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import pandas as pd
from utils.helper_funcs import spaceToDepth

from sklearn.model_selection import train_test_split

from imgaug import augmenters as iaa


In [2]:
print(tf.__version__)

2.3.1


# Preprocessing and Dataset Generation

In [3]:
# Load Train and val data. Similar to HW2, task 4, we used Keras ImageDataGenerator class to load train generator

# Subfolder structure of TinyImagenet dataset contains images sorted into respective classes
# This allows us to easily call 'flow_from_directory', and directly apply preprocessing via keras datagenerator
train_datagen = ImageDataGenerator(rescale=1/255.0)
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', target_size=(32, 32), batch_size = 256, shuffle=True)
classes_lookup = train_generator.class_indices
print(classes_lookup)

Found 100000 images belonging to 200 classes.
{'n01443537': 0, 'n01629819': 1, 'n01641577': 2, 'n01644900': 3, 'n01698640': 4, 'n01742172': 5, 'n01768244': 6, 'n01770393': 7, 'n01774384': 8, 'n01774750': 9, 'n01784675': 10, 'n01855672': 11, 'n01882714': 12, 'n01910747': 13, 'n01917289': 14, 'n01944390': 15, 'n01945685': 16, 'n01950731': 17, 'n01983481': 18, 'n01984695': 19, 'n02002724': 20, 'n02056570': 21, 'n02058221': 22, 'n02074367': 23, 'n02085620': 24, 'n02094433': 25, 'n02099601': 26, 'n02099712': 27, 'n02106662': 28, 'n02113799': 29, 'n02123045': 30, 'n02123394': 31, 'n02124075': 32, 'n02125311': 33, 'n02129165': 34, 'n02132136': 35, 'n02165456': 36, 'n02190166': 37, 'n02206856': 38, 'n02226429': 39, 'n02231487': 40, 'n02233338': 41, 'n02236044': 42, 'n02268443': 43, 'n02279972': 44, 'n02281406': 45, 'n02321529': 46, 'n02364673': 47, 'n02395406': 48, 'n02403003': 49, 'n02410509': 50, 'n02415577': 51, 'n02423022': 52, 'n02437312': 53, 'n02480495': 54, 'n02481823': 55, 'n02486410'

In [4]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['Img', 'Label', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)
valid_datagen = ImageDataGenerator(rescale=1/255.0)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='Img', y_col='Label', batch_size=256, shuffle=True, class_mode='categorical', target_size=(64, 64))
print(validation_generator.class_indices)

Found 10000 validated image filenames belonging to 200 classes.
{'n01443537': 0, 'n01629819': 1, 'n01641577': 2, 'n01644900': 3, 'n01698640': 4, 'n01742172': 5, 'n01768244': 6, 'n01770393': 7, 'n01774384': 8, 'n01774750': 9, 'n01784675': 10, 'n01855672': 11, 'n01882714': 12, 'n01910747': 13, 'n01917289': 14, 'n01944390': 15, 'n01945685': 16, 'n01950731': 17, 'n01983481': 18, 'n01984695': 19, 'n02002724': 20, 'n02056570': 21, 'n02058221': 22, 'n02074367': 23, 'n02085620': 24, 'n02094433': 25, 'n02099601': 26, 'n02099712': 27, 'n02106662': 28, 'n02113799': 29, 'n02123045': 30, 'n02123394': 31, 'n02124075': 32, 'n02125311': 33, 'n02129165': 34, 'n02132136': 35, 'n02165456': 36, 'n02190166': 37, 'n02206856': 38, 'n02226429': 39, 'n02231487': 40, 'n02233338': 41, 'n02236044': 42, 'n02268443': 43, 'n02279972': 44, 'n02281406': 45, 'n02321529': 46, 'n02364673': 47, 'n02395406': 48, 'n02403003': 49, 'n02410509': 50, 'n02415577': 51, 'n02423022': 52, 'n02437312': 53, 'n02480495': 54, 'n02481823

# Network 1 Architecture

In [5]:
#Define models using Keras.Models similar to HW3 Task 2

# Note: Cannot use one input dims as input dims change throughout training and so does data. Perhaps need to fit a new model each time?
# InputDims = (64,64)
BatchSize = 256
NumChannels = 3

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Concatenate, GlobalAveragePooling2D, Lambda
from tensorflow.keras import Model

## Input
# inputs = Input(shape=(InputDims[0],InputDims[1],3))
inputs = Input(shape=(None, None,3)) # <- Use None to allow flexible input shape.

## Block 1: 5x Conv2D layers (32,64,128,256,512) with BatchNorm and Relu activation with final MaxPool2D

block1 = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(inputs) # <- Do not use activation as need to squeeze in BatchNorm
block1 = BatchNormalization()(block1)
block1 = Activation('relu')(block1)
block1 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block1) 
block1 = BatchNormalization()(block1)
block1 = Activation('relu')(block1)
block1 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block1) 
block1 = BatchNormalization()(block1)
block1 = Activation('relu')(block1)
block1 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block1) 
block1 = BatchNormalization()(block1)
block1 = Activation('relu')(block1)
block1 = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block1) 
block1 = BatchNormalization()(block1)
block1 = Activation('relu')(block1)

block1 = MaxPool2D(pool_size=(2,2), strides=2)(block1)

## Block 2: 5x Conv2D layers (64,128,256,512,1024) with BatchNorm and Relu activation with final MaxPool2D

block2 = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block1) 
block2 = BatchNormalization()(block2)
block2 = Activation('relu')(block2)
block2 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block2) 
block2 = BatchNormalization()(block2)
block2 = Activation('relu')(block2)
block2 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block2) 
block2 = BatchNormalization()(block2)
block2 = Activation('relu')(block2)
block2 = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block2) 
block2 = BatchNormalization()(block2)
block2 = Activation('relu')(block2)
block2 = Conv2D(filters=1024, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block2) 
block2 = BatchNormalization()(block2)
block2 = Activation('relu')(block2)

block2 = MaxPool2D(pool_size=(2,2), strides=2) (block2)
block1_split = spaceToDepth(block1)
# Need to call spaceToDepth via Lambda layer
# block1_split = Lambda(spaceToDepth)(block1)
#Call concatenate on split 
block2 = Concatenate()([block1_split,block2])

## Block 3: 5x Conv2D layers (32,128,256,512,1024) with BatchNorm and Relu activation with final MaxPool2D

block3 = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block2) 
block3 = BatchNormalization()(block3)
block3 = Activation('relu')(block3)
block3 = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block3) 
block3 = BatchNormalization()(block3)
block3 = Activation('relu')(block3)
block3 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block3) 
block3 = BatchNormalization()(block3)
block3 = Activation('relu')(block3)
block3 = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block3) 
block3 = BatchNormalization()(block3)
block3 = Activation('relu')(block3)
block3 = Conv2D(filters=1024, kernel_size=(3,3), strides=(1,1), padding='same', activation=None)(block3) 
block3 = BatchNormalization()(block3)
block3 = Activation('relu')(block3)

block3 = MaxPool2D(pool_size=(2,2), strides=2) (block3)
block2_split = spaceToDepth(block2)
# block2_split = Lambda(spaceToDepth)(block2)
#Call concatenate on split 
block3 = Concatenate()([block2_split,block3])

## Block 4: Final Conv2d (200) and GlobalAveragePool2D
block4 = Conv2D(filters=200, kernel_size=(1,1), strides=(1,1), padding='same', activation=None)(block3) 
block4 = GlobalAveragePooling2D(data_format=None)(block4)

## Output
output = Activation('softmax')(block4)

model1 = Model(inputs=[inputs], outputs=[output])
model1.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [6]:
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
# Callback ReduceLROnPlateau
# Double check: Original paper did not specify specific values so implementation will use default values
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, verbose=0, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0)
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau

In [21]:
batch_size=256
model1.fit(train_generator, 
           epochs=15, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator,  
           verbose=1, 
           callbacks=[reduceLR])

model1.save('Models/Model_1.h5')  

Epoch 1/15
390/390 [==============================] - 2074s 5s/step - loss: 5.2888 - accuracy: 0.0348 - val_loss: 5.6413 - val_accuracy: 0.0118 - lr: 0.0010
Epoch 2/15
390/390 [==============================] - 614s 2s/step - loss: 4.1799 - accuracy: 0.1135 - val_loss: 4.5713 - val_accuracy: 0.0686 - lr: 0.0010
Epoch 3/15
390/390 [==============================] - 615s 2s/step - loss: 3.7159 - accuracy: 0.1827 - val_loss: 4.7265 - val_accuracy: 0.0961 - lr: 0.0010
Epoch 4/15
390/390 [==============================] - 615s 2s/step - loss: 3.3906 - accuracy: 0.2365 - val_loss: 4.3494 - val_accuracy: 0.1071 - lr: 0.0010
Epoch 5/15
390/390 [==============================] - 616s 2s/step - loss: 3.1462 - accuracy: 0.2804 - val_loss: 4.4644 - val_accuracy: 0.1078 - lr: 0.0010
Epoch 6/15
390/390 [==============================] - 618s 2s/step - loss: 2.9299 - accuracy: 0.3199 - val_loss: 4.1508 - val_accuracy: 0.1419 - lr: 0.0010
Epoch 7/15
390/390 [==============================] - 615s 2s/s

## Update target_size to 64x64

In [10]:
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', 
                                                    target_size=(64, 64), 
                                                    batch_size = 64, 
                                                    shuffle=True)

Found 100000 images belonging to 200 classes.


In [11]:
model = load_model('Models/Model_1.h5')

model.fit(train_generator, 
           epochs=15, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator, 
           verbose=1, 
           callbacks=[reduceLR])

model.save('Models/Model_2.h5')

Epoch 1/15
1562/1562 [==============================] - 2226s 1s/step - loss: 2.5410 - accuracy: 0.3928 - val_loss: 3.1819 - val_accuracy: 0.3000
Epoch 2/15
1562/1562 [==============================] - 2198s 1s/step - loss: 2.1290 - accuracy: 0.4754 - val_loss: 3.3033 - val_accuracy: 0.3053
Epoch 3/15
1562/1562 [==============================] - 2192s 1s/step - loss: 1.8778 - accuracy: 0.5296 - val_loss: 2.4640 - val_accuracy: 0.4290
Epoch 4/15
1562/1562 [==============================] - 2193s 1s/step - loss: 1.6731 - accuracy: 0.5750 - val_loss: 3.0181 - val_accuracy: 0.3663
Epoch 5/15
1562/1562 [==============================] - 2199s 1s/step - loss: 1.4832 - accuracy: 0.6149 - val_loss: 2.3023 - val_accuracy: 0.4632
Epoch 6/15
1562/1562 [==============================] - 2197s 1s/step - loss: 1.2994 - accuracy: 0.6576 - val_loss: 2.4213 - val_accuracy: 0.4564
Epoch 7/15
1562/1562 [==============================] - 2195s 1s/step - loss: 1.1095 - accuracy: 0.7018 - val_loss: 2.3428 -

## Update target size to 16x16

In [12]:
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', 
                                                    target_size=(16, 16), 
                                                    batch_size = 256, 
                                                    shuffle=True)

Found 100000 images belonging to 200 classes.


In [13]:
model = load_model('Models/Model_2.h5')

model.fit(train_generator, 
           epochs=10, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator, 
           verbose=1, 
           callbacks=[reduceLR])

model.save('Models/Model_3.h5')

Epoch 1/10
390/390 [==============================] - 247s 632ms/step - loss: 5.5033 - accuracy: 0.1135 - val_loss: 4.1946 - val_accuracy: 0.1388
Epoch 2/10
390/390 [==============================] - 241s 617ms/step - loss: 4.0901 - accuracy: 0.1506 - val_loss: 4.4587 - val_accuracy: 0.1113
Epoch 3/10
390/390 [==============================] - 241s 617ms/step - loss: 3.8522 - accuracy: 0.1805 - val_loss: 4.8705 - val_accuracy: 0.0976
Epoch 4/10
390/390 [==============================] - 241s 617ms/step - loss: 3.6561 - accuracy: 0.2065 - val_loss: 5.3165 - val_accuracy: 0.0839
Epoch 5/10
390/390 [==============================] - 241s 617ms/step - loss: 3.4872 - accuracy: 0.2309 - val_loss: 5.9440 - val_accuracy: 0.0731
Epoch 6/10
390/390 [==============================] - 240s 616ms/step - loss: 3.3343 - accuracy: 0.2535 - val_loss: 6.3609 - val_accuracy: 0.0680
Epoch 7/10
390/390 [==============================] - 241s 619ms/step - loss: 3.2023 - accuracy: 0.2758 - val_loss: 6.8745 -

## Update target size to 64x64


In [14]:
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', 
                                                    target_size=(64, 64), 
                                                    batch_size = 64, 
                                                    shuffle=True)

Found 100000 images belonging to 200 classes.


In [15]:
model = load_model('Models/Model_3.h5')

model.fit(train_generator, 
           epochs=15, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator, 
           verbose=1, 
           callbacks=[reduceLR])

model.save('Models/Model_4.h5')

Epoch 1/15
1562/1562 [==============================] - 2203s 1s/step - loss: 0.2309 - accuracy: 0.9461 - val_loss: 2.4107 - val_accuracy: 0.5387
Epoch 2/15
1562/1562 [==============================] - 2205s 1s/step - loss: 0.0477 - accuracy: 0.9953 - val_loss: 2.4927 - val_accuracy: 0.5406
Epoch 3/15
1562/1562 [==============================] - 2201s 1s/step - loss: 0.0279 - accuracy: 0.9983 - val_loss: 2.5770 - val_accuracy: 0.5469
Epoch 4/15
1562/1562 [==============================] - 2200s 1s/step - loss: 0.0194 - accuracy: 0.9990 - val_loss: 2.6493 - val_accuracy: 0.5440
Epoch 5/15
1562/1562 [==============================] - 2195s 1s/step - loss: 0.0147 - accuracy: 0.9993 - val_loss: 2.7203 - val_accuracy: 0.5429
Epoch 6/15
1562/1562 [==============================] - 2196s 1s/step - loss: 0.0117 - accuracy: 0.9994 - val_loss: 2.7841 - val_accuracy: 0.5448
Epoch 7/15
1562/1562 [==============================] - 2197s 1s/step - loss: 0.0098 - accuracy: 0.9996 - val_loss: 2.8390 -

### Image augmentation

Indirect approach: 
images with 32x32 resolutions for the beginning few epochs 
followed by 64x64 resolution images. 
Then we fed images with
16x16 resolutions for the next few numbers of epochs
Finally fed the model with 64x64 resolutions images

In [8]:
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', 
                                                    target_size=(16, 16), 
                                                    batch_size = 64, 
                                                    shuffle=True)

Found 100000 images belonging to 200 classes.


In [9]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, 
                                                         directory='./tiny-imagenet-200/val/images/', 
                                                         target_size=(16, 16),
                                                         color_mode='rgb', 
                                                         class_mode='categorical', 
                                                         x_col='Img', 
                                                         y_col='Label',
                                                         batch_size=256, 
                                                         shuffle=True)


Found 10000 validated image filenames belonging to 200 classes.


In [19]:
model = load_model('Models/Model_4.h5')

model.fit(train_generator, 
           epochs=5, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator, 
           verbose=1, 
           callbacks=[reduceLR])

model.save('Models/Model_5.h5')

Epoch 1/5
1562/1562 [==============================] - 223s 143ms/step - loss: 8.0533 - accuracy: 0.1851 - val_loss: 6.2752 - val_accuracy: 0.1669
Epoch 2/5
1562/1562 [==============================] - 221s 141ms/step - loss: 4.6288 - accuracy: 0.2013 - val_loss: 4.7239 - val_accuracy: 0.1655
Epoch 3/5
1562/1562 [==============================] - 221s 141ms/step - loss: 3.9333 - accuracy: 0.2076 - val_loss: 4.2704 - val_accuracy: 0.1714
Epoch 4/5
1562/1562 [==============================] - 220s 141ms/step - loss: 3.6765 - accuracy: 0.2229 - val_loss: 4.0738 - val_accuracy: 0.1822
Epoch 5/5
1562/1562 [==============================] - 221s 141ms/step - loss: 3.5239 - accuracy: 0.2378 - val_loss: 3.9487 - val_accuracy: 0.1948


In [10]:
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', 
                                                    target_size=(64, 64), 
                                                    batch_size = 64, 
                                                    shuffle=True)

Found 100000 images belonging to 200 classes.


In [11]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, 
                                                         directory='./tiny-imagenet-200/val/images/', 
                                                         target_size=(64, 64),
                                                         color_mode='rgb', 
                                                         class_mode='categorical', 
                                                         x_col='Img', 
                                                         y_col='Label',
                                                         batch_size=256, 
                                                         shuffle=True)

Found 10000 validated image filenames belonging to 200 classes.


In [14]:
model = load_model('Models/Model_5.h5')

model.fit(train_generator, 
           epochs=5, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator, 
           verbose=1, 
           callbacks=[reduceLR])

model.save('Models/Model_6.h5')

Epoch 1/5
1562/1562 [==============================] - 2224s 1s/step - loss: 0.2026 - accuracy: 0.9577 - val_loss: 2.5346 - val_accuracy: 0.5133
Epoch 2/5
1562/1562 [==============================] - 2196s 1s/step - loss: 0.0395 - accuracy: 0.9969 - val_loss: 2.5944 - val_accuracy: 0.5260
Epoch 3/5
1562/1562 [==============================] - 2199s 1s/step - loss: 0.0179 - accuracy: 0.9992 - val_loss: 2.6657 - val_accuracy: 0.5362
Epoch 4/5
1562/1562 [==============================] - 2202s 1s/step - loss: 0.0102 - accuracy: 0.9997 - val_loss: 2.7151 - val_accuracy: 0.5393
Epoch 5/5
1562/1562 [==============================] - 2198s 1s/step - loss: 0.0074 - accuracy: 0.9996 - val_loss: 2.7461 - val_accuracy: 0.5398


In [12]:
seq = iaa.SomeOf((0, None), [
    iaa.Affine(scale=(0.8, 1.2)),   #scale the image 
    iaa.CoarseDropout((0.0, 0.3), size_percent=(0.05, 0.07)),   #CoarseDropout 
    iaa.Affine(rotate=45),          #Rotate the image
    iaa.AdditiveGaussianNoise(scale=0.1*255),  #Add Gaussiance Noise
    iaa.CropAndPad(percent=(-0.3, 0.3))   # CropAndPad
     ])

 Medium article: https://towardsdatascience.com/data-augmentation-for-deep-learning-4fe21d1a4eb9
 SomeOf: https://imgaug.readthedocs.io/en/latest/source/overview/meta.html#someof

In [13]:
train_datagen = ImageDataGenerator(rescale=1/255.0, preprocessing_function=seq.augment_image )
train_generator = train_datagen.flow_from_directory('./tiny-imagenet-200/train', target_size=(64, 64),\
                                                    batch_size = 256, shuffle=True)

Found 100000 images belonging to 200 classes.


In [14]:
valid_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = valid_datagen.flow_from_dataframe(val_data, 
                                                         directory='./tiny-imagenet-200/val/images/', 
                                                         target_size=(64, 64),
                                                         color_mode='rgb', 
                                                         class_mode='categorical', 
                                                         x_col='Img', 
                                                         y_col='Label',
                                                         batch_size=64, 
                                                         shuffle=True)

Found 10000 validated image filenames belonging to 200 classes.


In [15]:
model = load_model('Models/Model_6.h5')

model.fit(train_generator, 
           epochs=5, 
           steps_per_epoch=train_generator.n // train_generator.batch_size, 
           validation_steps=validation_generator.n // validation_generator.batch_size, 
           validation_data=validation_generator, 
           verbose=1, 
           callbacks=[reduceLR])

model.save('Models/Model_7.h5')

Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[256,512,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/conv2d_24/Conv2D (defined at <ipython-input-15-868754b6ac1d>:9) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_5504]

Function call stack:
train_function


In [ ]:
model.predict(
    train_generator, batch_size=None
)

# Resources

"Space-to-depth" implementation

https://www.tensorflow.org/api_docs/python/tf/nn/space_to_depth

For loading data

https://keras.io/api/preprocessing/image/#imagedatagenerator-class

Regularizers (For Network 2)

https://keras.io/api/layers/regularizers/